In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve
from sklearn import svm

import matplotlib.pyplot as plt
import matplotlib.cm as colormap
plt.rcParams['image.cmap'] = 'Paired'

import numpy as np
np.random.seed(1)

from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.xception import Xception,preprocess_input

# Disable GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/malimg_paper_dataset_imgs"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:       Adialer.C	Number of images: 122
Label: 1	Family:       Agent.FYI	Number of images: 116
Label: 2	Family:       Allaple.A	Number of images: 2949
Label: 3	Family:       Allaple.L	Number of images: 1591
Label: 4	Family:   Alueron.gen!J	Number of images: 198
Label: 5	Family:       Autorun.K	Number of images: 106
Label: 6	Family:     C2LOP.gen!g	Number of images: 200
Label: 7	Family:         C2LOP.P	Number of images: 146
Label: 8	Family:  Dialplatform.B	Number of images: 177
Label: 9	Family:       Dontovo.A	Number of images: 162
Label:10	Family:        Fakerean	Number of images: 381
Label:11	Family:   Instantaccess	Number of images: 431
Label:12	Family:      Lolyda.AA1	Number of images: 213
Label:13	Family:      Lolyda.AA2	Number of images: 184
Label:14	Family:      Lolyda.AA3	Number of images: 123
Label:15	Family:       Lolyda.AT	Number of images: 159
Label:16	Family:     Malex.gen!J	Number of images: 136
Label:17	Family:   Obfuscator.AD	Number of images: 142
Label:18

In [4]:
X.shape

(9339, 224, 224, 3)

In [5]:
# Encoding classes (y) into integers (y_encoded) and then generating one-hot-encoding (Y)
encoder = LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
Y = np_utils.to_categorical(y_encoded)

In [6]:
# Creating base_model (Xception notop)
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = Xception(weights='imagenet', input_shape=image_shape, include_top=False)

In [7]:
filename = 'malimg-xceptionfeatures.npy'
if os.path.exists(filename):
    print("Loading Xception extracted features from %s ..." %(filename))
    xceptionfeatures = np.load(filename)
else:
    print("Extracting features from Xception layers ...")
    xceptionfeatures = base_model.predict(X)
    print("Saving Xception extracted features into %s ..." %(filename))
    np.save(filename, xceptionfeatures)

Loading Xception extracted features from malimg-xceptionfeatures.npy ...


In [8]:
xceptionfeatures.shape

(9339, 7, 7, 2048)

In [9]:
xceptionfeatures = np.reshape(xceptionfeatures,(xceptionfeatures.shape[0],-1))

In [10]:
xceptionfeatures.shape

(9339, 100352)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=cv, verbose=10, n_jobs=5)
top_model.fit(xceptionfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] ........................... C=0.01, score=0.984475, total=16.8min
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.981799, total=17.4min
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.989293, total=18.0min


[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed: 18.4min


[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.982334, total=17.9min
[CV] C=0.1 ...........................................................
[CV] ........................... C=0.01, score=0.985011, total=18.6min
[CV] C=0.1 ...........................................................
[CV] ............................ C=0.1, score=0.982869, total=15.3min
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.989293, total=16.4min
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.982869, total=17.6min


[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed: 36.3min


[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.984475, total=18.1min
[CV] C=1.0 ...........................................................
[CV] ............................ C=0.1, score=0.984475, total=17.9min
[CV] C=1.0 ...........................................................
[CV] ............................ C=1.0, score=0.982869, total=13.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.989293, total=15.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.984475, total=15.7min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.982869, total=16.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.984475, total=16.3min


...

[CV] C=1000000000.0 ..................................................
[CV] ..................... C=10000000.0, score=0.984475, total=12.2min
[CV] C=1000000000.0 ..................................................
[CV] .................... C=100000000.0, score=0.984475, total=11.0min
[CV] C=1000000000.0 ..................................................
[CV] .................... C=100000000.0, score=0.982869, total=11.1min
[CV] C=1000000000.0 ..................................................
[CV] ................... C=1000000000.0, score=0.989293, total=10.0min
[CV] C=1000000000.0 ..................................................
[CV] ................... C=1000000000.0, score=0.982869, total=11.3min
[CV] C=10000000000.0 .................................................
[CV] .................... C=100000000.0, score=0.984475, total=12.4min
[CV] C=10000000000.0 .................................................
[CV] ................... C=1000000000.0, score=0.982869, total=10.6min
[CV] C

[Parallel(n_jobs=5)]: Done  63 out of  65 | elapsed: 170.5min remaining:  5.4min


[CV] .................. C=10000000000.0, score=0.982869, total=10.8min
[CV] .................. C=10000000000.0, score=0.984475, total=11.2min


[Parallel(n_jobs=5)]: Done  65 out of  65 | elapsed: 172.4min finished


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=1, test_size=0.2,
            train_size=None),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.10000000000000001} - score: 0.9848


In [13]:
top_model.cv_results_

{'mean_fit_time': array([ 1062.50610785,  1022.95291152,   921.08120556,   732.3861938 ,
          704.31550484,   704.16542249,   699.08809419,   695.34806442,
          687.55341482,   694.57284393,   670.4588994 ,   662.18448362,
          654.23602881]),
 'mean_score_time': array([ 1.52441072,  1.42248025,  1.43411589,  1.24696035,  1.20887294,
         1.2084269 ,  1.16443262,  1.22446041,  1.1944839 ,  1.29539185,
         1.2336906 ,  1.24393678,  1.13680658]),
 'mean_test_score': array([ 0.98458244,  0.98479657,  0.98479657,  0.98479657,  0.98479657,
         0.98479657,  0.98479657,  0.98479657,  0.98479657,  0.98479657,
         0.98479657,  0.98479657,  0.98479657]),
 'mean_train_score': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 1000000000.0 10000000000.0],
              mask = [False False False False False False

In [ ]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.98458, std: 0.00265, params: {'C': 0.01},
 mean: 0.98480, std: 0.00236, params: {'C': 0.10000000000000001},
 mean: 0.98480, std: 0.00236, params: {'C': 1.0},
 mean: 0.98480, std: 0.00236, params: {'C': 10.0},
 mean: 0.98480, std: 0.00236, params: {'C': 100.0},
 mean: 0.98480, std: 0.00236, params: {'C': 1000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 10000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 100000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 1000000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 10000000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 100000000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 1000000000.0},
 mean: 0.98480, std: 0.00236, params: {'C': 10000000000.0}]

In [ ]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
svm_model = svm.LinearSVC(C=top_model.best_params_['C'])
train_sizes = np.linspace(.1, 1.0, 10)
train_sizes, train_scores, test_scores = learning_curve(svm_model, xceptionfeatures, y, cv=cv, n_jobs=-1, train_sizes=train_sizes)

In [ ]:
figure = plt.gcf()
figure.set_size_inches(24, 9)
skplot.learning_curve(train_scores, test_scores, train_sizes)
plt.show()